In [1]:
import requests, json
import pandas as pd
pd.options.display.max_rows = 999

import time, os
import numpy as np

import random
import time

import requests
import json
from bs4 import BeautifulSoup
import PyDST

In [2]:


url='https://www.jobindex.dk/jobsoegning.json?q=&jobage=archive&mindate=20070822&maxdate=20090822&page=2'
response = requests.get(url)
html=response.json()['result_list_box_html']
soup = BeautifulSoup(html,'lxml')
table_node = soup.find('div', attrs={'class':'results component--default'}) # find table node with specific attribute
table_node
#data.soup.find('div', attrs={'id':'results'})

<div class="results component--default">
<div data-beacon-tid="r3234069"><div class="jix_robotjob jix_job_archived">
<strong>Efterskolelærer søges hurtigst muligt!</strong>
<br/>
    Efterskolelærer søges hurtigst muligt!
    <br/>
<b>Staby Efterskole</b>, <b>Holstebro</b>
<br/>
<cite>Jobzonen, 22. august 2009</cite>
<div style="clear:both;"></div>
<span class="jix_toolbar jix_appetizer_toolbar" id="jix_toolbar_r3234069">
<ul class="toolbar-line toolbar-line-primary">
<li class="toolbar-myjob toolbar-myjob-notapplied">
<a data-action="add-dialog" href="/bruger/dine-job/r3234069/gem" rel="nofollow">
<span class="visible-xs-inline">Gem</span>
<span class="hidden-xs">Gem job</span>
</a> </li>
<li class="toolbar-follow hidden-xs jix_companyid_27331 jix_companyindex_follow">
<a class="follow_on" href="/bruger/dine-virksomheder?companyid=27331" rel="nofollow">Følg</a>
<a class="follow_off" href="/bruger/dine-virksomheder/27331/slet" rel="nofollow">Følger</a>
</li> <li class="toolbar-share"><

In [3]:
type(firm_city)

NameError: name 'firm_city' is not defined

In [ ]:
jobtitle_egne=table_node.find_all('a')
print(jobtitle_egne)

In [ ]:
jobtitle=table_node.find_all('strong')
citation=table_node.find_all('cite')
firm_city=table_node.find_all('b')
print(firm_city)

#kode til importerede annoncer for firma og by.

firm=[]
city=[]
cc=0

for i in firm_city:
    if cc %2==0:
        firm.append(i.text)
    else :
        city.append(i.text)
    cc += 1
print(firm)
print(city)

# BNP data

In [ ]:
conn = PyDST.connection(language = 'en')

In [ ]:
res = conn.get_tables('06')
res.info

In [ ]:
conn.get_metadata('NKN1').unit

In [ ]:
resp = conn.get_data(
                table_id = 'NKN1',
                variables = ['transakt', 'prisenhed', 'sæson', 'tid'])
#data = resp.df

In [ ]:
data = resp.df
data

In [ ]:
data['PRISENHED'].unique()

In [ ]:
BNP=data[(data['PRISENHED']=='2010-prices, chained values, (bill. DKK.)') & 
     (data['TRANSAKT']=='B.1*g Gross domestic product') & 
     (data['SÆSON']=='Seasonally adjusted') ]

In [ ]:
BNP.columns=['TRANSAKT', 'PRICE UNIT', 'SEASON', 'TIME', 'GDP, BILLIONS OF DKK']

In [ ]:
BNP

In [ ]:
BNP['YEAR']=BNP['TIME'].apply(lambda x: int(x.split('Q')[0]))


In [ ]:
BNP['QUARTER']=BNP['TIME'].apply(lambda x: int(x.split('Q')[1]))

In [ ]:
BNP

In [ ]:
BNP[BNP['YEAR']>= 2008].to_csv('test.csv')

In [ ]:
BNP[BNP['YEAR'].isin([2008,2009])]

# Beskæftigelsesdata

In [ ]:
resp_1 = conn.get_data(
                table_id = 'AKU103',
                variables = ['beskstatus', 'tid'])



In [ ]:
data_1 = resp_1.df
data_1

In [ ]:
conn.get_metadata('AKU103').variables

In [ ]:
data_1.columns=['EMPLOY_STATUS', 'TIME', 'EMPLOYED, (1000 per.)']

In [ ]:
EMPLOY=data_1[(data_1['EMPLOY_STATUS']=='Employed') ]

In [ ]:
EMPLOY['YEAR']=EMPLOY['TIME'].apply(lambda x: int(x.split('Q')[0]))

In [ ]:
EMPLOY['QUARTER']=EMPLOY['TIME'].apply(lambda x: int(x.split('Q')[1]))

In [ ]:
EMPLOY

In [ ]:
EMPLOY[EMPLOY['YEAR']>= 2008].to_csv('test_employ.csv')

In [ ]:
EMPLOY[EMPLOY['YEAR'].isin([2008,2009])]

In [ ]:
EMPLOY.drop(['EMPLOY_STATUS', 'TIME'], axis=1,inplace=True)

# BAROMETER for industrien

In [ ]:
conn.get_metadata('BARO8').variables

In [ ]:
resp_2 = conn.get_data(
                table_id = 'BARO8',
                variables = ['indikator', 'sæson', 'tid'])


In [ ]:
data_2 = resp_2.df
data_2

In [ ]:
BAR_IND=data_2[(data_2['INDIKATOR']=='Confidence indicator, total') &
               (data_2['SÆSON']=='Seasonally adjusted')]

In [ ]:
BAR_IND['YEAR']=BAR_IND['TID'].apply(lambda x: int(x.split('M')[0]))
BAR_IND['QUARTER']=BAR_IND['TID'].apply(lambda x: int(x.split('M')[1]))

In [ ]:
BAR_IND.drop(['INDIKATOR', 'SÆSON'], axis=1,inplace=True)


In [ ]:
BAR_IND

In [ ]:
BAR_IND.columns=['TIME', 'CONFIDENCE_IND_TOTAL_IND', 'YEAR', 'QUARTER']
BAR_IND.to_csv('test_bar_ind.csv')

# BAROMETER for Bygge og anlægsvirksomhed

In [ ]:
conn.get_metadata('KBYG44').variables


In [ ]:
resp_3 = conn.get_data(
                table_id = 'KBYG44',
                variables = ['indikator', 'sæson', 'tid'])

In [ ]:
data_3 = resp_3.df
data_3

In [ ]:
BAR_BYG=data_3[(data_3['INDIKATOR']=='Confidence indicator, total') &
               (data_3['SÆSON']=='Seasonally adjusted')]

In [ ]:
BAR_BYG['YEAR']=BAR_BYG['TID'].apply(lambda x: int(x.split('M')[0]))
BAR_BYG['QUARTER']=BAR_BYG['TID'].apply(lambda x: int(x.split('M')[1]))

In [ ]:
BAR_BYG

In [ ]:
BAR_BYG.drop(['INDIKATOR', 'SÆSON'], axis=1,inplace=True)

In [ ]:
BAR_BYG.columns=['TIME', 'CONFIDENCE_IND_TOTAL_BYG', 'YEAR', 'QUARTER']
BAR_BYG

In [ ]:
BAR_BYG.to_csv('test_bar_byg.csv')

# BAROMETER for ... service og detail går kun tilbage til 2011

In [ ]:
conn.get_metadata('KBYG44').variables

In [ ]:
resp_4 = conn.get_data(
                table_id = 'KBYG44',
                variables = ['indikator', 'sæson', 'tid'])

In [ ]:
data_3 = resp_3.df
data_3

In [ ]:
BAR_BYG=data_3[(data_3['INDIKATOR']=='Confidence indicator, total') &
               (data_3['SÆSON']=='Seasonally adjusted')]

In [ ]:
BAR_BYG['YEAR']=BAR_BYG['TID'].apply(lambda x: int(x.split('M')[0]))
BAR_BYG['QUARTER']=BAR_BYG['TID'].apply(lambda x: int(x.split('M')[1]))

In [ ]:
BAR_BYG.drop(['INDIKATOR', 'SÆSON'], axis=1,inplace=True)

In [ ]:
BAR_BYG.columns=['TIME', 'CONFIDENCE_IND_TOTAL_BYG', 'YEAR', 'QUARTER']
BAR_BYG

In [ ]:
BAR_BYG.to_csv('test_bar_byg.csv')